In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

/home/heekue83/miniconda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = load_dataset("GAIR/lima", split="train")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

Generating test split: 100%|██████████| 300/300 [00:00<00:00, 121327.86 examples/s]
tokenizer_config.json: 100%|██████████| 26.0/26.0 [00:00<00:00, 139kB/s]
config.json: 100%|██████████| 665/665 [00:00<00:00, 4.16MB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 3.16MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.88MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 3.32MB/s]


In [6]:
context_length = 128

def tokenize(element):
    lines = ["\n".join(x) for x in element["conversations"]]

    outputs = tokenizer(
        lines,
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = ds.map(
    tokenize, batched=True, remove_columns=ds.column_names, load_from_cache_file=False
)
tokenized_datasets

Map:   0%|          | 0/1030 [00:00<?, ? examples/s]

Map: 100%|██████████| 1030/1030 [00:01<00:00, 734.53 examples/s]


Dataset({
    features: ['input_ids'],
    num_rows: 4817
})

In [7]:
tokenized_datasets.column_names

['input_ids']